### Detect Sound Using Arduino and KY-038 sensor

<img src="Images/Cover_page.png" width="500" align="left" />

#### Overview

1. Prelude
2. Setting Up the Microcontroller 
3. Setting Up the VB.Net GUI
4. Demonstration
5. References

***

#### 1. Prelude

The KY-038 sensor has two outputs: analog output, A0, (i.e., output voltage signal) and digital output, DO (i.e., high and low signal). The threshold-sensitivity of the D0 signal can be adjusted via potentiometer on the sensor [2]. Here, I demonstrate how to read the D0 signal and plot the data using VB.Net GUI.  

#### 2. Setting Up the Microcontroller [1-3]

Parts:
- KY-038 sound sensor - 1x
- LED (any colors) - 1x
- a resistor (220 Oms) - 1x
- Arduino Uno - 1x

Connections between sound sensor and Arduino:
    - A0 of sensor connects to A0 of Arduino
    - G of sensor connects to GND of Arduino
    - + of sensor connects to 5V of Arduino
    - D0 of sensor connects to a resistor, which is also connected to pin 13 on one end and connects to the long leg of an LED. The short leg of the LED is connected to GND of Arduino. 

See below for the schematics and Arduino codes.
<br>

<p align="center"> <b> Figure 1. Schematics for when led is (a) off and (b) on. </b> </p>
<img src="Images/Fig_1.png" align="left" alt="" />


In [ ]:
//Code is taken from: https://tkkrlab.nl/wiki/Arduino_KY-038_Microphone_sound_sensor_module

int sensorPin = A0; // select the input pin for the potentiometer
int ledPin = 13; // select the pin for the LED
int sensorValue = 0; // variable to store the value coming from the sensor
int val = 0;// define numeric variables val
 
void setup () 
{
  pinMode (ledPin, OUTPUT);
  Serial.begin (9600);
}
 
void loop () 
{
  sensorValue = analogRead (sensorPin);
  //digitalWrite (ledPin, HIGH);
  //delay (sensorValue);
  //digitalWrite (ledPin, LOW);
  //delay (sensorValue);

  //this value determines the state of the led
  val = digitalRead(ledPin);
  
  //Serial.print (sensorValue, DEC); Serial.print(",");//write the sensorvalue
  //Serial.println (val); //
  Serial.println(sensorValue);
  
  //Adjust the sensorValue to the desired sensitivity  
  if (val==LOW && sensorValue>=24){
    digitalWrite(ledPin, HIGH);}
  else if (val==HIGH && sensorValue>=24){
    digitalWrite(ledPin, LOW);}
}

#### 3. 3. Setting Up the VB.Net GUI

To view the sound signal, we can either use the serial monitor or serial plotter on Arduino software or using VB.Net. To view the signal using Arduino software. Hook up Arduino Uno via USB port. On the software, go to Tools. Make sure that Arduino/Genuine Uni is selected as the Board. Also, make sure that the Arduino/Genuine Uno COM port is selected. If so, either click on Serial Monitor or Serial Plotter to view the streaming data. 

To view the signal using VB.Net, first we prepare the form (<b>Figure 2</b>).

<p align="center"> <b> Figure 2. VB.Net form design. </b> </p>
<img src="Images/Fig_2.png" align="left" alt="" />

Next, we write the VB.Net script (see below).

In [ ]:
Imports System.IO
Imports System.IO.Ports
Imports System.Windows.Forms.DataVisualization.Charting


Public Class frmMain
    'Code is adapted from: http://www.martyncurrey.com/arduino-and-visual-basic-part-1-receiving-data-from-the-arduino/

    Dim WithEvents SerialPort As New IO.Ports.SerialPort
    Dim selPort As String

    Dim comPORT As String
    Dim receivedData As String = ""

    Dim x As Integer


    Private Sub Form1_Load(sender As Object, e As EventArgs) Handles MyBase.Load

        Timer1.Enabled = False

        'set graph type and axes labels
        chtChart.Series(0).ChartType = SeriesChartType.Line
        chtChart.ChartAreas("ChartArea1").AxisX.Title = "Counts"
        chtChart.ChartAreas("ChartArea1").AxisY.Title = "Signal"
        chtChart.ChartAreas("ChartArea1").AxisY.Minimum = 0
        chtChart.ChartAreas("ChartArea1").AxisY.Maximum = 30

        comPORT = "" 'initialize comPORT as null
        For Each sp As String In My.Computer.Ports.SerialPortNames
            cmbComPort.Items.Add(sp)
        Next
        Try
            'Use the first entry as the default value
            cmbComPort.Text = cmbComPort.Items(0)
        Catch ex As Exception
            MsgBox("No ports are detected. Please check connections.", MsgBoxStyle.Exclamation, "Warning")
        End Try

    End Sub

    Public Sub SPSetup()    'Serial Port Setup
        On Error Resume Next
        If SerialPort.IsOpen Then
            SerialPort.Close()
        End If

        'Set communication parameters
        SerialPort.PortName = selPort
        SerialPort.BaudRate = 9600
        SerialPort.DataBits = 8
        SerialPort.StopBits = IO.Ports.StopBits.One
        SerialPort.Handshake = IO.Ports.Handshake.None
        SerialPort.Parity = IO.Ports.Parity.None

        SerialPort.Open() 'connect to Serial Port

    End Sub
    Private Sub btnExit_Click(sender As Object, e As EventArgs) Handles btnExit.Click

        Me.Close() 'close form

    End Sub
    Private Sub cmbComPort_SelectedIndexChanged(sender As Object, e As EventArgs) Handles cmbComPort.SelectedIndexChanged

        'set comPort variable to user selected value
        If (cmbComPort.SelectedItem <> "") Then
            comPORT = cmbComPort.SelectedItem
        End If

    End Sub
    Private Sub btnConnect_Click(sender As Object, e As EventArgs) Handles btnConnect.Click

        'If communication is off, start it and begin communication
        If (btnConnect.Text = "Connect") Then
            If (comPORT <> "") Then
                SerialPort1.Close()
                SerialPort1.PortName = comPORT
                SerialPort1.BaudRate = 9600
                SerialPort1.DataBits = 8
                SerialPort1.Parity = Parity.None
                SerialPort1.StopBits = StopBits.One
                SerialPort1.Handshake = Handshake.None
                SerialPort1.Encoding = System.Text.Encoding.Default
                SerialPort1.ReadTimeout = 10000

                SerialPort1.Open()

                btnConnect.Text = "Disconnect"
                Timer1.Enabled = True
                lblTimer.Text = "Timer: ON"

                'initialize the value of x
                x = 0
                'clear graph
                chtChart.Series(0).Points.Clear()

            Else 'If not connected, provide a warning
                MsgBox("Select a COM port first")
            End If
            'Close port and stop timer

        Else
            'Send command to Arduino to deactivate DHT11
            SerialPort1.Write("0")

            'Close the serial port
            SerialPort1.Close()

            btnConnect.Text = "Connect"
            Timer1.Enabled = False
            lblTimer.Text = "Timer: OFF"

        End If

    End Sub
    Private Sub Timer1_Tick(sender As Object, e As EventArgs) Handles Timer1.Tick

        'As long as timer is on, send command to Arduino to activate DHT11
        SerialPort1.Write("1")

        Dim filePath As String = String.Format("V_{0}.txt", DateTime.Today.ToString("dd-MMM-yyyy"))

        'Obtain data via serial communication and update information in textbox
        receivedData = ReceiveSerialData()

        'save data to notepad with today's date
        'ref.: https://stackoverflow.com/questions/6959977/writing-a-txt-file-without-a-extra-line

        Try

            'https://social.msdn.microsoft.com/Forums/vstudio/en-US/17ab29b7-1492-4de1-8693-a8f65e24ec98/drawing-a-graph-with-real-time-data?forum=vbgeneral
            chtChart.Series(0).Points.AddXY(x, receivedData) 'display time in seconds
            'Chart1.ChartAreas(0).AxisX.CustomLabels.Add(x - 1, x + 1, Now.ToString("HH:mm:ss fff"))
            x += 1

            txtBox.Text = x & ", " & receivedData

            'save data into a notepad
            Using writer As New StreamWriter(filePath, True)
                'record data, including date and time
                'writer.Write(DateTime.Now & ", " & receivedData)
                writer.Write(x & "," & receivedData)

            End Using

        Catch ex As Exception
            'Do nothing
        End Try

    End Sub
    Function ReceiveSerialData() As String

        'read data via serial communication
        Dim Incoming As String
        Try
            'check for incoming signals
            Incoming = SerialPort1.ReadExisting()

            If Incoming Is Nothing Then
                Return "nothing" & vbCrLf
            Else
                Return Incoming
            End If

        Catch ex As TimeoutException
            'Return "Error: Serial Port read timed out."
            Return ""
        End Try

    End Function

End Class

#### 4. Demonstration

To view sound signal detected by KY-038 sensor, hook up the Arduino/K-038 to a computer via USB cable. Then, upload the code via Arduino sofware (see <b> section 2</b> above for the Arduino code). Run the VB.Net program. On the VB.Net form, selet the appropriate COM port and click connect to view the sound signal (<b>Figure 3</b>). A practical application to this setup is to replace the LED light with a device that can be turned on and off using a sound. An appropriate threshold signal (i.e., sensorValue) must be set in the Arduino code.

<p align="center"> <b> Figure 3. Sound signal measured by KY-038 sound sensor. </b> </p>
<img src="Images/Fig_3.png" align="left" alt="" />

#### 5. References

<li> 1.  http://henrysbench.capnfatz.com/henrys-bench/arduino-sensors-and-input/arduino-sound-detection-sensor-tutorial-and-user-manual/
<li> 2. https://tkkrlab.nl/wiki/Arduino_KY-038_Microphone_sound_sensor_module
<li> 3. https://www.youtube.com/watch?v=YK_xnMlzFh8